This notebook demonstrates how to solve the problem of **Named Entity Disambiguation**, which is linking a mention of an entity (like "Michael Jordan") in a text to the correct entry in a knowledge base (like Wikipedia). It explores two common techniques: String Similarity and Cosine Similarity.

In [ ]:
# Import the 'wikipedia' library to fetch content from Wikipedia pages.
import wikipedia
# Import 'spacy' for natural language processing tasks, like identifying entities.
import spacy
# Import 'numpy' for efficient numerical operations, especially with vectors.
import numpy as np
# Import 'Counter' from the collections module for counting items (though not used in the final code, it's common in NLP).
from collections import Counter
# Import the 'math' module for basic mathematical functions.
import math

### Initialize the NLP Model

Here, we load a pre-trained English language model from the `spaCy` library. We disable the 'parser' component because we only need the model for tokenization and named entity recognition (NER), which makes it load faster.

In [ ]:
# Load the 'en' model from spaCy. This model is trained on English text.
# We disable the 'parser' to make the loading and processing faster since we don't need dependency parsing.
nlp = spacy.load('en', disable=['parser'])
# This is a common workaround if the standard 'en' model link is not installed.
# It explicitly loads a small English web corpus.
# nlp = spacy.load('en_core_web_sm', disable=['parser'])

### Fetch Candidate Wikipedia Pages

Our goal is to figure out which "Michael Jordan" a text is referring to. We'll use Wikipedia as our knowledge base. First, we select the three most likely candidates and download their summary content using the `wikipedia` library. This content will serve as the context for each entity.

In [ ]:
# A dictionary to map the official Wikipedia page titles to a more descriptive 'type' for easy reference.
jordan_type = {
    'Michael Jordan': 'NBA',
    'Michael B. Jordan': 'actor',
    'Michael I. Jordan': 'professor'
}

# A list of the exact Wikipedia page titles for our candidate entities.
page_titles = ['Michael Jordan', 'Michael B. Jordan', 'Michael I. Jordan']

# Create an empty list to store the data we download from Wikipedia.
pages = []
# Loop through each title in our list of page titles.
for title in page_titles:
    # Print a message to show the progress.
    print("downloading '{}'".format(title))
    # Fetch the Wikipedia page object using the title.
    page = wikipedia.page(title)
    # Append a tuple containing the page's title and its summary text to our 'pages' list.
    pages.append((page.title, page.summary))

### Load Text Data for Disambiguation

Now, we need some example sentences that mention "Michael Jordan." These sentences are stored in a CSV file. The function below reads this file, and for each sentence (line), it uses `spaCy` to process it into a `doc` object. This `doc` object contains tokens and recognized entities, which we will use later.

In [ ]:
def read_data(filename):
    
    """ Read data from file 
    Input: 
        - filename containing one document per line
    Output:
        - a list of spaCy tokenized documents
    """
    
    # Initialize an empty list to store the processed documents.
    data=[]
    # Open the specified file for reading.
    with open(filename) as file:
        # Iterate over each line in the file.
        for line in file:
            # Process the line with our nlp model to create a spaCy doc object.
            # .strip() removes any leading/trailing whitespace.
            doc = nlp(line.strip())
            # Add the processed doc to our data list.
            data.append(doc)
    
    # Print how many documents were loaded.
    print("loaded {} docs".format(len(data)))
    
    # Return the list of spaCy documents.
    return data

# Call the function to read our test data from the specified CSV file.
docs = read_data("../data/MichaelJordan.csv")

### Q1. String Similarity

Our first approach is a simple one: compare the text of the entity found in our sentence (e.g., "Michael Jordan") directly with the titles of our candidate Wikipedia pages (e.g., "Michael Jordan," "Michael B. Jordan"). We'll use the **Jaccard Similarity** metric, which measures the overlap between two sets. For strings, it's the ratio of the number of common words to the total number of unique words in both strings.

In [ ]:
def string_similarity(str1, str2):
    """
    Return a string similarity score betweein 0 and 1, 
    where 1 indicates exact match, 0 indicates no match
    
    e.g. string_similarity('hello world', 'hello world') = 1
    """
    
    # This implements word-level Jaccard Similarity.
    # Convert the first string into a set of unique words.
    str1 = set(str1.split())
    # Convert the second string into a set of unique words.
    str2 = set(str2.split())
    # Calculate the intersection (common words) and union (total unique words).
    # The similarity is the size of the intersection divided by the size of the union.
    return float(len(str1 & str2)) / len(str1 | str2)

#### Check the String Similarity Function

Let's test our `string_similarity` function with a few examples to make sure it's working as expected.

In [ ]:
def check_string_similarity(str1, str2):
    # A helper function to print the comparison and the resulting similarity score.
    print("'{}' vs. '{}' = {}".format(str1, str2, string_similarity(str1, str2)))
    
# Test case 1: Identical strings should return a score of 1.0.
check_string_similarity("hello world", "hello world")
# Test case 2: Similar strings. They share "Michael" and "Jordan".
check_string_similarity("Michael B. Jordan", "Michael I. Jordan")
# Test case 3: Partially similar strings. They share "Clinton".
check_string_similarity("Hillary Clinton", "Bill Clinton")
# Test case 4: Completely different strings should return a score of 0.0.
check_string_similarity("one two three", "four five six")

### Rank Candidates by String Similarity

This function iterates through each of our test sentences (`docs`). For each sentence, it finds all named entities (like "Michael Jordan"). It then calculates the string similarity between each entity and each candidate Wikipedia page title. Finally, it selects the Wikipedia page with the highest similarity score as the best match for that sentence.

In [ ]:
def rank_string_similarity(docs, pages):
    """
    For each doc return the Wiki page that is the best match based on string similarity.
    Returns a list of (title, score) tuples representing the best match for each doc.
    """
    
    # Initialize an empty list to store the results for each document.
    result = []
    # Loop through each document in our test data.
    for doc in docs:
        # Initialize the best score to -1 (so any valid score will be higher).
        best_score = -1
        # Initialize the best page to the first page by default.
        best_page = pages[0]
        
        # doc.ents contains all named entities recognized by spaCy in the document.
        for e in doc.ents:
            # For each entity, compare it against all candidate Wikipedia pages.
            for page in pages:
                # Calculate the string similarity between the entity's text and the page's title.
                sim = string_similarity(e.text, page[0])
                # If this similarity score is the highest we've seen so far for this doc...
                if sim > best_score:
                    # ...update the best score.
                    best_score = sim
                    # ...and update the best page.
                    best_page = page
        
        # After checking all entities and pages, append the best match (title and score) to our results.
        result.append((best_page[0], best_score))
    # Return the list of best matches.
    return result
    
def display_result(closest, docs):
    
    """ Print results in a readable format """

    # Iterate through the documents with an index.
    for i in range(len(docs)):
        # Print the original document text.
        print(docs[i])
        # Get the title of the best-matching Wikipedia page from the results.
        title = closest[i][0]
        # Print the best match title, its type (e.g., 'NBA'), and the similarity score.
        print("{} - {} ({})".format(title, jordan_type[title], closest[i][1]))
        # Print a blank line for better readability.
        print()      

#### Evaluate String Similarity Performance

Now let's run the ranking and see the results. As you'll see, this method works well when the entity mention is an exact match for a page title, but it fails when the context is needed for disambiguation (like in the "computer science" example).

In [ ]:
# Run the ranking function to get the best matches based on string similarity.
string_rank = rank_string_similarity(docs, pages)
# Display the results in a formatted way.
display_result(string_rank, docs)

### Q2. Cosine Similarity

String similarity alone isn't enough because it ignores context. A better approach is to compare the **meaning** of the text surrounding the entity with the meaning of the Wikipedia page summary. We can do this using **word embeddings** (like GloVe) to convert text into numerical vectors. Then, we can calculate the **cosine similarity** between these vectors. A high cosine similarity (close to 1) means the vectors point in a similar direction, implying similar semantic content.

First, we load pre-trained GloVe word embeddings. These embeddings map words to high-dimensional vectors.

In [ ]:
def load_embeddings(filename, max_vocab_size):
    """
    Load embeddings from a file.
    Returns a numpy array of embeddings and a vocabulary dictionary {word: index}.
    """
    
    # Initialize a dictionary to map words to their index in the embeddings list.
    vocab={}
    # Initialize a list to hold the embedding vectors.
    embeddings=[]
    # Open the embeddings file.
    with open(filename) as file:
        
        # The first line of the w2v format file contains the number of words and the vector size.
        cols=file.readline().split(" ")
        num_words=int(cols[0])
        size=int(cols[1])
        # Add a zero vector for padding (index 0).
        embeddings.append(np.zeros(size))
        # Add a zero vector for unknown words (UNK) (index 1).
        embeddings.append(np.zeros(size))
        # Add special tokens to our vocabulary.
        vocab["_0_"]=0
        vocab["_UNK_"]=1
        
        # Iterate over each line in the file, where each line is a word and its vector.
        for idx,line in enumerate(file):

            # Stop if we reach the maximum desired vocabulary size.
            if idx+2 >= max_vocab_size:
                break

            # Split the line into the word and the vector values.
            cols=line.rstrip().split(" ")
            # Convert the vector values to a numpy array of floats.
            val=np.array(cols[1:])
            # The first column is the word.
            word=cols[0]
            
            # Add the vector to our embeddings list.
            embeddings.append(val)
            # Add the word and its index to our vocabulary. The index is idx+2 due to padding/UNK tokens.
            vocab[word]=idx+2

    # Convert the list of embeddings to a single numpy array for efficiency.
    return np.array(embeddings), vocab

# Load the top 50,000 word embeddings from the GloVe file.
embeddings, vocab=load_embeddings("../data/glove.42B.300d.50K.w2v.txt", 50000)

#### One-Time Conversion of GloVe file

This cell is a utility script that only needs to be run once. The raw GloVe file format is slightly different from the `word2vec` format that our `load_embeddings` function expects. This code uses the `gensim` library to perform the conversion.

In [ ]:
"""
Run after downloading glove.42B.300d.50K.txt for the first time.
Only run this cell if you got a 'file not found' error above.
"""
# Import libraries from gensim for handling word vectors.
from gensim.models import Word2Vec, KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

# Define the path to the raw GloVe file.
glove_file="../data/glove.42B.300d.50K.txt"
# Define the path for the output file in word2vec format.
glove_in_w2v_format="../data/glove.42B.300d.50K.w2v.txt"
# Run the conversion utility. The underscore '_' is used to ignore the function's return value.
_ = glove2word2vec(glove_file, glove_in_w2v_format)

# Now, load the embeddings from the newly created w2v-formatted file.
embeddings, vocab=load_embeddings("../data/glove.42B.300d.50K.w2v.txt", 50000)

### Create Document Vector Representations

To compare two pieces of text (like a sentence and a Wikipedia summary), we need to represent each one as a single vector. A simple and effective way to do this is to get the embedding for every word in the text and then calculate the average of all those vectors. This function does exactly that.

In [ ]:
def get_doc_representation(doc, vocab, embeddings):
    """
    Return one vector that represents the entire array of input tokens
    by averaging the word embeddings of all words in the document.
    """
    
    # This implementation takes the average of all word embeddings in the doc.
    # Create an empty list to store the embedding vectors of words found in the vocab.
    words = []
    # Iterate through each token (word) in the document.
    for token in doc:
        # Check if the lowercase version of the token exists in our vocabulary.
        if token.lower() in vocab:
            # If it exists, get its vector from the embeddings matrix and add it to our list.
            words.append(embeddings[vocab[token.lower()]].astype(np.float))
    # Calculate the mean of all vectors along axis 0 (column-wise average).
    # This results in a single vector representing the entire document.
    return np.mean(words, axis=0)

### Rank Candidates by Cosine Similarity

This is the main function for our second approach. It works as follows:
1. It first converts each candidate Wikipedia page's summary into a single vector representation using our `get_doc_representation` function.
2. Then, it iterates through each of our test sentences (`docs`).
3. For each sentence, it also creates a single vector representation.
4. It then calculates the cosine similarity between the sentence's vector and each of the Wikipedia page vectors.
5. The page with the highest cosine similarity score is chosen as the best match.

In [ ]:
def cos_similarity(v1, v2):
    """ Returns cosine similarity between two vectors """
    # Calculate the dot product of the two vectors.
    # Divide by the product of the magnitudes (norms) of the vectors.
    cos = np.dot(v1, v2) / (np.sqrt(np.dot(v1,v1)) * np.sqrt(np.dot(v2,v2)))
    return cos

def rank_cos_similarity(docs, pages, vocab, embeddings):
    """
    For each doc return the Wiki page that is the best match based on cosine similarity.
    """
    
    # Initialize an empty list to store the final results.
    result = []
    
    # Pre-calculate the vector representations for each Wikipedia page summary.
    page_representations = []
    for p in pages:
        # The summary text is the second element in the tuple.
        summary = p[1]
        # Get the vector representation for the summary. Note: summary is split into words first.
        emb = get_doc_representation(summary.split(" "), vocab, embeddings)
        # Store the page title along with its vector representation.
        page_representations.append((p[0], emb))
    
    # Now, process each input document.
    for doc in docs:     
        # Get the vector representation for the entire spaCy doc.
        # We extract the text of each token before passing it to the function.
        doc_representation = get_doc_representation([token.text for token in doc], vocab, embeddings)
        
        # A list to store the similarity scores for the current doc against all pages.
        scores = []
        
        # Iterate through the pre-calculated page representations.
        for p in page_representations:
            title = p[0] # The page title
            emb = p[1]   # The page's vector
            # Calculate the cosine similarity between the doc vector and the page vector.
            sim = cos_similarity(doc_representation, emb)
            # Append the title and its similarity score to the scores list.
            scores.append((p[0], sim))

        # Sort the scores in descending order based on the similarity score (the second element, x[1]).
        scores = sorted(scores, key=lambda x: x[1], reverse=True)
    
        # The best match is the first item in the sorted list. Append it to the results.
        result.append(scores[0])
    return result

#### Evaluate Cosine Similarity Performance

Let's execute the cosine similarity ranking. Observe the results. The context of "computer science" in the first sentence now correctly helps the model link "Michael Jordan" to the professor, demonstrating that this method is much more powerful than simple string matching.

In [ ]:
# Run the ranking function using cosine similarity.
closest_cos = rank_cos_similarity(docs, pages, vocab, embeddings)
# Display the results using the same helper function.
display_result(closest_cos, docs)

### Q3. Other Features?

Based on your results above, can you think of other features that might help improve entity linking performance further? Name one feature and justify why you think it is a good feature.

One powerful feature would be **Popularity** or **Priors**. This feature represents the overall probability that a mention of a name (like "Michael Jordan") refers to a specific entity, regardless of the context. For example, the NBA player Michael Jordan is globally far more famous than the professor or the actor. We could determine this by looking at Wikipedia page view counts or the number of incoming links to each page. By incorporating this as a feature, if the context is ambiguous, the model can default to the most probable entity. This would be combined with the context similarity score (like cosine similarity) to make a final, more robust decision. For instance, the final score could be a weighted average: `Score = 0.7 * CosineSimilarity + 0.3 * PopularityScore`.